In [194]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz
import difflib 
from fuzzywuzzy import process
import time
from tqdm import tqdm 
import fitz

from functools import reduce

### importing books

In [342]:
vol1_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf'
vol2_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf'
vol3_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf'

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

vol1_pages = [vol1_doc[i] for i in range(vol1_doc.page_count)]
vol2_pages = [vol2_doc[i] for i in range(vol2_doc.page_count)]
vol3_pages = [vol3_doc[i] for i in range(vol3_doc.page_count)]

In [196]:
vol1_char_df = pd.read_pickle("../input/char_df/vol1_df.pkl")
vol2_char_df = pd.read_pickle("../input/char_df/vol2_df.pkl")
vol3_char_df = pd.read_pickle("../input/char_df/vol3_df.pkl")

In [230]:
#mout. index: 
vol1_index_path = '../output/local/index_output/vol1_nonitalics.csv'
vol2_index_path = '../output/local/index_output/vol2_nonitalics.csv'
vol3_index_path = '../output/local/index_output/vol3_nonitalics.csv'

vol1_index_df = pd.read_csv(vol1_index_path)
vol2_index_df = pd.read_csv(vol2_index_path)
vol3_index_df = pd.read_csv(vol3_index_path)

#changing name of columns of mout. indecies 
vol1_index_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)
vol2_index_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)
vol3_index_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)

In [231]:
vol1_word_df = vol1_char_df.loc[:, ['vol_num', 'page_num', 
                                    'block_num', 'block_num_absolute', 'block_bbox',
                                    'line_num', 'line_wmode', 'line_dir', 'line_bbox', 
                                    'span_num', 'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender', 'span_descender', 'span_origin', 'span_bbox', 
                                    'word_num', 'word','word_bbox', 'pruned_word', 'pruned_word_bbox']].drop_duplicates()

vol2_word_df = vol2_char_df.loc[:, ['vol_num', 'page_num', 
                                    'block_num', 'block_num_absolute', 'block_bbox',
                                    'line_num', 'line_wmode', 'line_dir', 'line_bbox', 
                                    'span_num', 'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender', 'span_descender', 'span_origin', 'span_bbox', 
                                    'word_num', 'word','word_bbox', 'pruned_word', 'pruned_word_bbox']].drop_duplicates()

vol3_word_df = vol3_char_df.loc[:, ['vol_num', 'page_num', 
                                    'block_num', 'block_num_absolute', 'block_bbox',
                                    'line_num', 'line_wmode', 'line_dir', 'line_bbox', 
                                    'span_num', 'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender', 'span_descender', 'span_origin', 'span_bbox', 
                                    'word_num', 'word','word_bbox', 'pruned_word', 'pruned_word_bbox']].drop_duplicates()

In [384]:
vol1_word_df.columns

Index(['vol_num', 'page_num', 'block_num', 'block_num_absolute', 'block_bbox',
       'line_num', 'line_wmode', 'line_dir', 'line_bbox', 'span_num',
       'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender',
       'span_descender', 'span_origin', 'span_bbox', 'word_num', 'word',
       'word_bbox', 'pruned_word', 'pruned_word_bbox', 'word_pairs',
       'possible_index_match', 'match_score', 'single_match',
       'single_match_score', 'word_triplets', 'triple_match',
       'triple_match_score', 'word_quads', 'quad_match', 'quad_match_score'],
      dtype='object')

### getting word pairs

*right now last word of page i matches with alst word of page i + 1*

things that I need to account for and am not doing so right now: 
- what if a word is in the next paragraph/block 
- what if a word is in the next page 

In [388]:
df = vol1_word_df.copy()
n = 2
out_col = f"{n}_words"
line_group_cols = ['vol_num', 'page_num', 
                    'block_num', 'block_num_absolute', 'block_bbox', 
                    'line_num', 'line_wmode', 'line_dir', 'line_bbox']

word_tuple_dict = {1: df['word'].tolist()}
list(zip(vol1_word_df['word'], new_word))
df[out_col] = df.groupby(line_group_cols)['word'].shift(-1)
df



,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,match_score,single_match,single_match_score,word_triplets,triple_match,triple_match_score,word_quads,quad_match,quad_match_score,2_words
0,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,NaN,AUTRES OUVRAGES DU,NaN,NaN,AUTRES OUVRAGES DU MÊME,NaN,NaN,OUVRAGES
6,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,NaN,OUVRAGES DU MÊME,NaN,NaN,OUVRAGES DU MÊME AUTEUR,NaN,NaN,DU
14,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,NaN,DU MÊME AUTEUR,NaN,NaN,DU MÊME AUTEUR La,NaN,NaN,MÊME
16,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,NaN,MÊME AUTEUR La,NaN,NaN,MÊME AUTEUR La végétation,NaN,NaN,AUTEUR
20,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,NaN,NaN,NaN,AUTEUR La végétation,NaN,NaN,AUTEUR La végétation arborescente,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761774,1,641,4,6,"(154.0800018310547, 284.6300048828125, 227.289...",0,0,"(1.0, 0.0)","(154.0800018310547, 284.6300048828125, 227.289...",0,...,NaN,NaN,NaN,JUILLET MIL NEUF,NaN,NaN,JUILLET MIL NEUF CENT,NaN,NaN,NaN
1761781,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,NaN,NaN,NaN,MIL NEUF CENT,NaN,NaN,MIL NEUF CENT SOIXANTE-SIX,NaN,NaN,NEUF
1761784,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,NaN,NaN,NaN,NEUF CENT SOIXANTE-SIX,NaN,NaN,NEUF CENT SOIXANTE-SIX None,NaN,NaN,CENT
1761788,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,NaN,NaN,NaN,CENT SOIXANTE-SIX None,NaN,NaN,CENT SOIXANTE-SIX None None,NaN,NaN,SOIXANTE-SIX


In [ ]:
# def flags_decomposer(flags):
#     """Make font flags human readable."""
#     l = []
#     if flags & 2 ** 0:
#         l.append("superscript")
#     if flags & 2 ** 1:
#         l.append("italic")
#     if flags & 2 ** 2:
#         l.append("serifed")
#     else:
#         l.append("sans")
#     if flags & 2 ** 3:
#         l.append("monospaced")
#     else:
#         l.append("proportional")
#     if flags & 2 ** 4:
#         l.append("bold")
#     return ", ".join(l)

In [396]:
6 | 7 
# 100 | 101 = 111 
(4 | 5) & 2 ** 1

0

In [428]:
# from functools import reduce

# # example list of numbers
# numbers = [4,1010]

# # apply bitwise OR operation to all elements
# result = reduce(lambda x, y: x | y, numbers)

# print([bin(el) for el in numbers], bin(result))

# f_list = [(4, 4),
#  (4, 4),
#  (4, 4),
#  (4, 4),
#  (4, 0),
#  (6, 6),
#  (6, 6),
#  (6, 6),
#  (6, 6),
#  (6, 6)]
# list(map(lambda flag_list : reduce(lambda x, y: x | y, flag_list), f_list))

In [429]:
def get_n_words_flagged(df, n, inplace = True):
    #assumes n > 1
    out_words_col = f"{n}_words"
    out_flags_col = f"{n}_flags"
    
    line_group_cols = ['vol_num', 'page_num', 
                       'block_num', 'block_num_absolute', 'block_bbox', 
                       'line_num', 'line_wmode', 'line_dir', 'line_bbox']
    
    n_words_lists = [None for i in range(n)]
    n_words_flags = [None for i in range(n)]

    n_words_lists[0] = df['word']
    n_words_flags[0] = df['span_flags']

    for i in range(1, n):
        n_words_lists[i] = df.groupby(line_group_cols)['word'].shift(-i, fill_value="")
        n_words_flags[i] = df.groupby(line_group_cols)['span_flags'].shift(-i, fill_value=0)

    zip_n_words = list(zip(*n_words_lists))
    n_word_string = list(map(lambda n_word_list : " ".join(n_word_list), zip_n_words))

    zip_n_flags = list(zip(*n_words_flags))
    combine_flags = list(map(lambda flag_list : reduce(lambda x, y: x | y, flag_list), zip_n_flags))
    
    if inplace == True:
        df[out_words_col] = n_word_string
        df[out_flags_col] = combine_flags
    return n_word_string, combine_flags

In [232]:
new_word = vol1_word_df['word'].shift(-1)
# create a new column with pairs of consecutive words
word_pairs_list = list(zip(vol1_word_df['word'], new_word))
word_pairs = list(map(lambda x: f"{x[0]} {x[1]}", word_pairs_list))
vol1_word_df['word_pairs'] = word_pairs

In [238]:
vol1_index_df

,Unnamed: 0,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon_rank,taxon_rank_detailed
0,1,NaN,cilicica,NaN,Ant. et Ky,epithet,epithet
1,5,Acanthophyllum,NaN,NaN,NaN,genus,genus
2,6,Acanthophyllum,kurdicum,NaN,Boiss. et Hausskn.,epithet,epithet
3,23,Achyranthes,NaN,NaN,NaN,genus,genus
4,24,Achyranthes,aspera,NaN,L.,epithet,epithet
...,...,...,...,...,...,...,...
1383,5473,Vulpia,uniglumis,NaN,(Sol.) Dumort.,epithet,epithet
1384,5481,Zannichellia,NaN,NaN,NaN,genus,genus
1385,5482,Zannichellia,palustris,NaN,L.,epithet,epithet
1386,5495,Zostera,NaN,NaN,NaN,genus,genus


In [250]:
#list of genera from index
vol1_genera = vol1_index_df[vol1_index_df['taxon_rank'] == 'genus']['mouterde_genus'].str.upper().tolist()

#list of species binomial from main text
vol1_species_temp_df = vol1_index_df[(vol1_index_df['taxon_rank'] == 'epithet') & (~vol1_index_df['mouterde_genus'].isna())]
vol1_species_binomial_list = list(zip(vol1_species_temp_df['mouterde_genus'], vol1_species_temp_df['mouterde_epithet']))
vol1_species = list(map(lambda x: f"{x[0]} {x[1]}", vol1_species_binomial_list))
vol1_species_abriviation = list(map(lambda x: f"{x[0][0]}. {x[1]}", vol1_species_binomial_list))

In [251]:
vol1_genera

['ACANTHOPHYLLUM',
 'ACHYRANTHES',
 'ADIANTUM',
 'AEGILOPS',
 'AEGILOTRICUM',
 'AELLENIA',
 'AELUROPUS',
 'AGROPYRQN',
 'AGROSTEMMA',
 'AGROSTIS',
 'AIRA',
 'AIZOON',
 'ALISMA',
 'ALLIUM',
 'ALNUS',
 'ALOE',
 'ALOPECURUS',
 'ALTERNANTHERA',
 'AMARANTHUS',
 'AMMOCHLOA',
 'AMMOPHILA',
 'ANABASIS',
 'ANACAMPTIS',
 'ANDROCYMBIUM',
 'ANDROPOGON',
 'ANKYROPETALUM',
 'ANTHOXANTHUM',
 'ANTINORIA',
 'APERA',
 'ARCEUTHOBIUM',
 'ARCEUTHOS',
 'ARENARIA',
 'ARISARUM',
 'ARISTIDA',
 'ARISTOLOCHIA',
 'ARRHENATHERUM',
 'ARTHROCNEMUM',
 'ARUM',
 'ARUNDO',
 'ASPARAGUS',
 'ASPHODELINE',
 'ASPHODELUSI',
 'ASPLENIUM',
 'ATHYRIUM',
 'ATRAPHAXIS',
 'ATRIPLEX',
 'AVENA',
 'BASSIA',
 'BECKMANNIA',
 'BELLEVALIA',
 'BETA',
 'BIARUM',
 'BILDERDYKIA',
 'BLECHNUM',
 'BLYSMUS',
 'BOERHAVIA',
 'BOISSIERA',
 'BOTHRIOCHLOA',
 'BRACHIARIA',
 'BRACHYPODIUM',
 'BRIZA',
 'BROMUS',
 'BUFFONIA',
 'BUTOMUS',
 'CALAMAGROSTIS',
 'CALLIGONUM',
 'CANNABIS',
 'CAREX',
 'CARPINUS',
 'CATABROSA',
 'CATAPODIUM',
 'CEDRUS',
 'CELTIS',

In [252]:
def difflib_closest_matches(input_str, matching_lst):
    matched_list = difflib.get_close_matches(input_str, matching_lst)
    if len(matched_list) > 0:
        return matched_list[0]
    else:
        #print(input_str)
        return np.NaN

In [253]:
tqdm.pandas()

In [254]:
result = vol1_word_df['word_pairs'].progress_apply(lambda x: difflib_closest_matches(x, vol1_species))

100%|██████████| 316313/316313 [15:11<00:00, 347.13it/s]


In [256]:
vol1_word_df['possible_index_match'] = result

In [327]:
def difflib_closest_match_score(input_str, match_str):
    if isinstance(match_str, str):
        input_str = input_str.replace(' ', '')
        match_str = match_str.replace(' ', '')
        score = difflib.SequenceMatcher(None, input_str.lower(), match_str.lower()).ratio()
        return score
    else:
        return np.NaN

In [328]:
#vol1_word_df[~result.isna()].head(50)
vol1_word_df['match_score'] = vol1_word_df.progress_apply(lambda r: difflib_closest_match_score(r['word_pairs'], r['possible_index_match']), axis = 1)

100%|██████████| 316313/316313 [00:01<00:00, 166534.18it/s]


### with single word matches

In [298]:
single_match = vol1_word_df['word'].progress_apply(lambda x: difflib_closest_matches(x, vol1_species))

100%|██████████| 316313/316313 [05:22<00:00, 981.52it/s] 


In [329]:
vol1_word_df['single_match'] = single_match
vol1_word_df['single_match_score'] = vol1_word_df.progress_apply(lambda r: difflib_closest_match_score(r['word'], r['single_match']), axis = 1)

100%|██████████| 316313/316313 [00:01<00:00, 178536.42it/s]


In [333]:
vol1_word_df[vol1_word_df['single_match_score'] > 0.95]
# have to make cut off high to eliminate junk ... but this may not be the best way ...

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,possible_index_match,match_score,single_match,single_match_score,word_triplets,triple_match,triple_match_score,word_quads,quad_match,quad_match_score
61222,1,24,16,28,"(72.0, 469.989990234375, 568.2545166015625, 48...",0,0,"(1.0, 0.0)","(72.0, 469.989990234375, 568.2545166015625, 48...",3,...,Stipa fontanesii,0.857143,Stipa fontanesii,1.000000,"Stipafontanesii ParL, Trisetaria",Stipa fontanesii,0.625000,"Stipafontanesii ParL, Trisetaria fiavescens",Trisetaria flavescens,0.625000
83427,1,32,18,34,"(61.439998626708984, 465.4700012207031, 572.49...",0,0,"(1.0, 0.0)","(61.439998626708984, 465.4700012207031, 572.49...",0,...,Iris pseudacorus,0.909091,Iris pseudacorus,1.000000,"Irispseudacorus L., se",Iris pseudacorus,0.789474,"Irispseudacorus L., se raréfient",Iris pseudacorus,0.625000
311662,1,119,14,30,"(48.47999954223633, 343.30999755859375, 410.82...",0,0,"(1.0, 0.0)","(48.47999954223633, 343.30999755859375, 410.82...",0,...,Sorghum halepen.se,0.864865,Sorghum halepen.se,0.969697,Sorghumhalepense (L.) Pers.,Sorghum halepen.se,0.711111,Sorghumhalepense (L.) Pers. —,Sorghum halepen.se,0.680851
324225,1,123,21,49,"(51.119998931884766, 473.1099853515625, 416.23...",0,0,"(1.0, 0.0)","(51.119998931884766, 473.1099853515625, 416.23...",0,...,Heteropogon f,0.846154,Heteropogon f,0.956522,Heteropogon con tort,Heteropogon contortus,0.926829,Heteropogon con tort us,Heteropogon contortus,0.954545
344988,1,131,6,15,"(54.720001220703125, 134.58999633789062, 418.4...",0,0,"(1.0, 0.0)","(54.720001220703125, 134.58999633789062, 418.4...",0,...,Stipa barbata,0.827586,Stipa barbata,1.000000,Stipabarbata Desf. (PI.,Stipa barbata,0.666667,"Stipabarbata Desf. (PI. XVIII,",NaN,NaN
384990,1,145,13,27,"(48.0, 224.989990234375, 410.60626220703125, 2...",0,0,"(1.0, 0.0)","(48.0, 224.989990234375, 410.60626220703125, 2...",0,...,Polypogon monspeliensis,0.916667,Polypogon monspeliensis,1.000000,Polypogonmonspeliensis (L.) Desf.—,Polypogon monspeliensis,0.771930,Polypogonmonspeliensis (L.) Desf.— Alopecurusm...,NaN,NaN
415645,1,156,23,49,"(40.560001373291016, 338.1499938964844, 403.57...",0,0,"(1.0, 0.0)","(40.560001373291016, 338.1499938964844, 403.57...",0,...,Trisetaria cavanillesii,0.862745,Trisetaria cavanillesii,1.000000,Trisetariacavanillesii (Trin.) Maire—,Trisetaria cavanillesii,0.733333,Trisetariacavanillesii (Trin.) Maire— Trisetum,Trisetaria cavanillesii,0.637681
685839,1,252,24,60,"(39.119998931884766, 393.5373840332031, 402.58...",0,0,"(1.0, 0.0)","(39.119998931884766, 393.5373840332031, 402.58...",0,...,Carex divulsa,0.774194,Carex divulsa,1.000000,Carexdivulsa Stoke˛— Carexmuricatah.,NaN,NaN,Carexdivulsa Stoke˛— Carexmuricatah. var.,NaN,NaN
1347059,1,483,8,14,"(60.959999084472656, 208.989990234375, 424.623...",0,0,"(1.0, 0.0)","(60.959999084472656, 208.989990234375, 424.623...",0,...,Rheum ribes,0.909091,Rheum ribes,1.000000,Rheumribes L. (PI.,Rheum ribes,0.689655,"Rheumribes L. (PI. CXXXIV,",NaN,NaN
1736072,1,626,68,113,"(229.1999969482422, 160.57000732421875, 408.58...",0,0,"(1.0, 0.0)","(229.1999969482422, 160.57000732421875, 280.62...",0,...,Heteropogon f,0.880000,Heteropogon f,0.956522,Heteropogon 46 f,Heteropogon f,0.896552,Heteropogon 46 f allionii,Heteropogon f,0.684211


In [309]:
second_word = vol1_word_df['word'].shift(-1)
third_word = vol1_word_df['word'].shift(-2)
# create a new column with pairs of consecutive words
word_triplets_list = list(zip(vol1_word_df['word'], second_word, third_word))
word_triplets = list(map(lambda x: f"{x[0]} {x[1]} {x[2]}", word_triplets_list))
vol1_word_df['word_triplets'] = word_triplets

In [313]:
triple_match = vol1_word_df['word_triplets'].progress_apply(lambda x: difflib_closest_matches(x, vol1_species))

100%|██████████| 316313/316313 [18:15<00:00, 288.81it/s]  


In [334]:
vol1_word_df['triple_match'] = triple_match
vol1_word_df['triple_match_score'] = vol1_word_df.progress_apply(lambda r: difflib_closest_match_score(r['word_triplets'], r['triple_match']), axis = 1)

100%|██████████| 316313/316313 [00:01<00:00, 170302.66it/s]


In [335]:
vol1_word_df[vol1_word_df['triple_match_score'] > .95]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,possible_index_match,match_score,single_match,single_match_score,word_triplets,triple_match,triple_match_score,word_quads,quad_match,quad_match_score
5287,1,4,26,60,"(72.95999908447266, 684.97998046875, 586.19470...",0,0,"(1.0, 0.0)","(74.87999725341797, 684.97998046875, 586.19470...",3,...,Rhynchospora glaucaVahl,0.900000,Rhynchospora glaucaVahl,0.705882,"Rhynchospora glauca Vahl,",Rhynchospora glaucaVahl,0.977778,"Rhynchospora glauca Vahl, retwuvées",Rhynchospora glaucaVahl,0.793103
22083,1,11,3,11,"(73.91999816894531, 99.70001220703125, 586.707...",0,0,"(1.0, 0.0)","(73.91999816894531, 99.70001220703125, 586.707...",4,...,NaN,NaN,NaN,NaN,de Boerhavia plumbaginea,Boerhavia plumbaginea,0.952381,de Boerhavia plumbaginea Cav.,Boerhavia plumbaginea,0.840000
27117,1,12,18,31,"(39.119998931884766, 600.5899658203125, 552.76...",3,0,"(1.0, 0.0)","(39.119998931884766, 647.3900146484375, 552.65...",0,...,NaN,NaN,NaN,NaN,de Cupressus sempervirens,Cupressus sempervirens,0.954545,"de Cupressus sempervirens L.,",Cupressus sempervirens,0.862745
38797,1,16,22,33,"(31.920000076293945, 672.3099975585938, 546.52...",2,0,"(1.0, 0.0)","(33.599998474121094, 703.510009765625, 315.213...",0,...,NaN,NaN,NaN,NaN,à Pinus brutia,Pinus brutia,0.956522,à Pinus brutia Ten.,Pinus brutia,0.774194
38956,1,16,25,37,"(33.599998474121094, 756.3699951171875, 545.92...",0,0,"(1.0, 0.0)","(33.599998474121094, 756.3699951171875, 545.92...",1,...,Osyris albaL.,0.909091,Osyris albaL.,0.666667,"Osyris alba L.,",Osyris albaL.,0.960000,"Osyris alba L., Laurus",Osyris albaL.,0.742857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757264,1,636,86,142,"(224.8800048828125, 535.074951171875, 308.8155...",1,0,"(1.0, 0.0)","(224.8800048828125, 544.6749877929688, 273.125...",0,...,Thelygonum cynocrambe,1.000000,Thelygonum cynocrambe,0.666667,Thelygonum cynocrambe L.,Thelygonum cynocrambe,0.952381,Thelygonum cynocrambe L. Themeda,Thelygonum cynocrambe,0.792453
1757760,1,637,11,19,"(41.040000915527344, 166.57000732421875, 156.0...",0,0,"(1.0, 0.0)","(42.2400016784668, 166.57000732421875, 156.053...",2,...,Trisetaria linearis,0.666667,NaN,NaN,Ky Trisetaria cavanillesii,Trisetaria cavanillesii,0.956522,Ky Trisetaria cavanillesii (Trin.),Trisetaria cavanillesii,0.807018
1759811,1,638,16,28,"(307.20001220703125, 88.33001708984375, 312.64...",0,0,"(1.0, 0.0)","(307.20001220703125, 88.33001708984375, 312.64...",0,...,Zannichellia palustris,0.705882,NaN,NaN,Z Zannichellia palustris,Zannichellia palustris,0.976744,Z Zannichellia palustris L.,Zannichellia palustris,0.897959
1759812,1,638,17,29,"(220.32000732421875, 97.65997314453125, 268.50...",0,0,"(1.0, 0.0)","(220.32000732421875, 97.65997314453125, 268.50...",0,...,Zannichellia palustris,1.000000,Zannichellia palustris,0.727273,Zannichellia palustris L.,Zannichellia palustris,0.954545,Zannichellia palustris L. Zea,Zannichellia palustris,0.862745


In [319]:
### quad match 
second_word = vol1_word_df['word'].shift(-1)
third_word = vol1_word_df['word'].shift(-2)
forth_word = vol1_word_df['word'].shift(-3)
# create a new column with pairs of consecutive words
word_quads_list = list(zip(vol1_word_df['word'], second_word, third_word, forth_word))
word_quads = list(map(lambda x: f"{x[0]} {x[1]} {x[2]} {x[3]}", word_quads_list))
vol1_word_df['word_quads'] = word_quads

In [320]:
quad_match = vol1_word_df['word_quads'].progress_apply(lambda x: difflib_closest_matches(x, vol1_species))

100%|██████████| 316313/316313 [56:46<00:00, 92.86it/s]   


In [336]:
vol1_word_df['quad_match'] = quad_match
vol1_word_df['quad_match_score'] = vol1_word_df.progress_apply(lambda r: difflib_closest_match_score(r['word_quads'], r['quad_match']), axis = 1)

100%|██████████| 316313/316313 [00:01<00:00, 179289.43it/s]


In [339]:
vol1_word_df[vol1_word_df['quad_match_score'] > .93]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,possible_index_match,match_score,single_match,single_match_score,word_triplets,triple_match,triple_match_score,word_quads,quad_match,quad_match_score
57703,1,23,15,36,"(85.68000030517578, 359.8135070800781, 599.282...",0,0,"(1.0, 0.0)","(86.4000015258789, 359.8135070800781, 599.2825...",0,...,NaN,NaN,NaN,NaN,d : 'Asphodeline,Asphodeline lutea,0.733333,d : 'Asphodeline damascena,Asphodeline damascena,0.930233
67176,1,26,13,21,"(55.68000030517578, 489.95001220703125, 575.16...",1,0,"(1.0, 0.0)","(55.68000030517578, 506.29998779296875, 565.91...",5,...,Celtis tournefortii,0.774194,NaN,NaN,Celtis Tourne- for,Celtis tournefortii,0.882353,Celtis Tourne- for tii,Celtis tournefortii,0.972973
272561,1,104,6,15,"(13.199999809265137, 101.3900146484375, 407.43...",0,0,"(1.0, 0.0)","(13.199999809265137, 101.3900146484375, 407.43...",1,...,Potamogeton lucens,0.903226,Potamogeton lucens,0.785714,Potamogeton lue en,Potamogeton lucens,0.909091,Potamogeton lue en s,Potamogeton lucens,0.941176
274184,1,104,28,65,"(44.15999984741211, 462.3500061035156, 407.814...",0,0,"(1.0, 0.0)","(44.15999984741211, 462.3500061035156, 407.814...",0,...,Potamogeton densus,0.903226,Potamogeton lucens,0.785714,Potamogeton den sus,Potamogeton densus,1.000000,Potamogeton den sus L.,Potamogeton densus,0.944444
275281,1,105,4,12,"(51.119998931884766, 150.54998779296875, 414.1...",0,0,"(1.0, 0.0)","(51.119998931884766, 150.54998779296875, 414.1...",0,...,Potamogeton panormitanus,0.820513,Potamogeton lucens,0.785714,Potamogeton panor mit,Potamogeton panormitanus,0.904762,Potamogeton panor mit anus,Potamogeton panormitanus,1.000000
282020,1,107,21,54,"(46.08000183105469, 526.6300048828125, 410.335...",0,0,"(1.0, 0.0)","(46.08000183105469, 526.6300048828125, 410.335...",0,...,Damasonium polyspermum,0.800000,Damasonium alisma,0.769231,Damasonium poly sper,Damasonium polyspermum,0.923077,Damasonium poly sper mum,Damasonium polyspermum,1.000000
287814,1,111,1,4,"(62.15999984741211, 50.07000732421875, 426.145...",0,0,"(1.0, 0.0)","(62.15999984741211, 50.07000732421875, 426.145...",0,...,NaN,NaN,NaN,NaN,Digit aria sanguinali,Digitaria sanguinalis,0.974359,Digit aria sanguinali s,Digitaria sanguinalis,1.000000
322409,1,123,5,12,"(51.36000061035156, 119.35000610351562, 415.10...",0,0,"(1.0, 0.0)","(51.36000061035156, 119.35000610351562, 415.10...",0,...,Andropogon distachyus,1.000000,Andropogon distachyus,0.666667,Andropogon distachyus L.,Andropogon distachyus,0.952381,Andropogon distachyus L. —,Andropogon distachyus,0.930233
324225,1,123,21,49,"(51.119998931884766, 473.1099853515625, 416.23...",0,0,"(1.0, 0.0)","(51.119998931884766, 473.1099853515625, 416.23...",0,...,Heteropogon f,0.846154,Heteropogon f,0.956522,Heteropogon con tort,Heteropogon contortus,0.947368,Heteropogon con tort us,Heteropogon contortus,1.000000
427415,1,160,30,86,"(41.52000045776367, 444.0574035644531, 406.680...",0,0,"(1.0, 0.0)","(41.52000045776367, 444.0574035644531, 406.680...",0,...,Aloe vera,0.615385,NaN,NaN,A vena pratensis,Avena pratensis,1.000000,A vena pratensis L.,Avena pratensis,0.933333


In [ ]:
### pent match


In [353]:
vol1_word_df[vol1_word_df['word_quads'] == 'Digit aria sanguinali s']
vol1_word_df[vol1_word_df['word'] == 'Pennisetum']

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,possible_index_match,match_score,single_match,single_match_score,word_triplets,triple_match,triple_match_score,word_quads,quad_match,quad_match_score
300771,1,115,24,57,"(64.80000305175781, 455.3900146484375, 437.058...",0,0,"(1.0, 0.0)","(64.80000305175781, 455.3900146484375, 437.058...",0,...,Penmsetum setaceum,0.914286,NaN,NaN,Pennisetum setaceum (Forsk.),Penmsetum setaceum,0.744186,Pennisetum setaceum (Forsk.) Chiov.,Penmsetum setaceum,0.653061
302863,1,116,15,34,"(41.7599983215332, 294.75, 405.2952575683594, ...",0,0,"(1.0, 0.0)","(41.7599983215332, 294.75, 405.2952575683594, ...",2,...,Penmsetum divisum,0.606061,NaN,NaN,Pennisetum ciliare (L.),NaN,NaN,Pennisetum ciliare (L.) Link,NaN,NaN


In [360]:
[s for s in vol1_species if s.split()[0] == 'Penmsetum']

['Penmsetum divisum', 'Penmsetum setaceum']

In [352]:
difflib_closest_match_score('Pennisetum divisum','Penmsetum divisum')

0.9090909090909091

In [377]:
# things to add: 
# make sure they're on the same line (fixing the shift situation by doing it on group by?)
# make sure single word match is nothing completely unreasonable (e.g > 0.1 or something like that)
# make sure no words in the grouping are italics 
likely_results = vol1_word_df[((vol1_word_df['match_score'] > 0.85) | 
                               (vol1_word_df['single_match_score'] > 0.9) | 
                               (vol1_word_df['triple_match_score'] > 0.9) | 
                               (vol1_word_df['quad_match_score'] > 0.9)) & 
                              (vol1_word_df['page_num'] < 616)]

In [378]:
likely_results

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,possible_index_match,match_score,single_match,single_match_score,word_triplets,triple_match,triple_match_score,word_quads,quad_match,quad_match_score
5262,1,4,26,60,"(72.95999908447266, 684.97998046875, 586.19470...",0,0,"(1.0, 0.0)","(74.87999725341797, 684.97998046875, 586.19470...",1,...,Colchicum libanoticum,1.000000,Colchicum tauri,0.782609,"Colchicum libanoticum Ehr.,",Colchicum libanoticum,0.888889,"Colchicum libanoticum Ehr., Rhynchospora",Colchicum libanoticum,0.701754
5287,1,4,26,60,"(72.95999908447266, 684.97998046875, 586.19470...",0,0,"(1.0, 0.0)","(74.87999725341797, 684.97998046875, 586.19470...",3,...,Rhynchospora glaucaVahl,0.900000,Rhynchospora glaucaVahl,0.705882,"Rhynchospora glauca Vahl,",Rhynchospora glaucaVahl,0.977778,"Rhynchospora glauca Vahl, retwuvées",Rhynchospora glaucaVahl,0.814815
6399,1,5,7,18,"(54.0, 238.25, 566.7366333007812, 269.48999023...",0,0,"(1.0, 0.0)","(54.0, 238.25, 566.1414794921875, 251.25)",2,...,Lycopodium cernuum,1.000000,Lycopodium cernuum,0.740741,"Lycopodium cernuum L.,",Lycopodium cernuum,0.918919,"Lycopodium cernuum L., Carex",Lycopodium cernuum,0.809524
6563,1,5,8,23,"(53.279998779296875, 273.97998046875, 566.9451...",0,0,"(1.0, 0.0)","(53.52000045776367, 273.97998046875, 566.24114...",3,...,Rhynchospora glaucaVahl,0.850000,Rhynchospora glaucaVahl,0.705882,"Rhynchospora giauca Vahl,",Rhynchospora glaucaVahl,0.933333,"Rhynchospora giauca Vahl, Daphne",Rhynchospora glaucaVahl,0.823529
6691,1,5,9,29,"(53.7599983215332, 310.46002197265625, 566.650...",0,0,"(1.0, 0.0)","(53.7599983215332, 310.46002197265625, 566.650...",0,...,Scirpus mucronatus,1.000000,Scirpus cernuus,0.666667,"Scirpus mucronatus L.,",Scirpus mucronatus,0.918919,"Scirpus mucronatus L., Gonringia",Scirpus mucronatus,0.739130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704888,1,612,4,8,"(36.959999084472656, 90.63497161865234, 288.37...",0,0,"(1.0, 0.0)","(36.959999084472656, 90.63497161865234, 288.37...",1,...,Iris bismarckiana,1.000000,NaN,NaN,Iris bismarckiana (même,Iris bismarckiana,0.864865,"Iris bismarckiana (même périodique,",Iris bismarckiana,0.666667
1706418,1,612,26,59,"(35.52000045776367, 328.31060791015625, 420.44...",0,0,"(1.0, 0.0)","(35.52000045776367, 328.31060791015625, 420.44...",5,...,Iris bismarckiana,0.866667,NaN,NaN,I. bismarckiana Regel,Iris bismarckiana,0.742857,I. bismarckiana Regel (The,Iris bismarckiana,0.666667
1707794,1,613,1,4,"(27.84000015258789, 44.40997314453125, 412.300...",0,0,"(1.0, 0.0)","(27.84000015258789, 44.40997314453125, 412.300...",2,...,NaN,NaN,NaN,NaN,le Rheum Ribes,Rheum ribes,0.909091,le Rheum Ribes (Thèse,Rheum ribes,0.714286
1707796,1,613,1,4,"(27.84000015258789, 44.40997314453125, 412.300...",0,0,"(1.0, 0.0)","(27.84000015258789, 44.40997314453125, 412.300...",3,...,Rheum ribes,1.000000,Rheum ribes,0.666667,Rheum Ribes (Thèse,Rheum ribes,0.769231,Rheum Ribes (Thèse présentée,NaN,NaN


In [379]:
from fitz.utils import getColor

In [380]:
# reset doc:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

In [381]:
for i in tqdm(likely_results.index):
    is_italic = bool(likely_results.loc[i, 'span_flags'] == 6)
    if not is_italic:
        page_num = int(likely_results.loc[i, 'page_num'])
        block_bbox = likely_results.loc[i, 'block_bbox']
        block_bbox = likely_results.loc[i, 'block_bbox']
        word_bbox = likely_results.loc[i, 'word_bbox']
        page = vol1_doc[page_num]
        r_high = fitz.Rect(word_bbox)
        r_box = fitz.Rect(block_bbox)
        annot_high = page.add_highlight_annot(r_high)
        annot_high.set_colors(stroke=getColor("antiquewhite"))
        annot_rect = page.add_rect_annot(r_box)
        annot_high.update()
        annot_rect.update()

marked_epithet_fname = "../output/local/main_text/fuzzyMatch95_vol1.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 1602/1602 [00:01<00:00, 812.07it/s]


In [272]:
vol1_word_df[vol1_word_df['word_pairs'] == 'Gypsophila frankenioides']

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_origin,span_bbox,word_num,word,word_bbox,pruned_word,pruned_word_bbox,word_pairs,possible_index_match,match_score
1648490,1,592,1,3,"(36.959999084472656, 52.3499755859375, 399.973...",0,0,"(1.0, 0.0)","(36.959999084472656, 52.3499755859375, 399.973...",0,...,"(36.959999084472656, 60.5999755859375)","(36.959999084472656, 52.3499755859375, 94.9909...",0,Gypsophila,"(36.959999084472656, 52.3499755859375, 94.9909...",Gypsophila,"(36.959999084472656, 52.3499755859375, 94.9909...",Gypsophila frankenioides,Gypsophila antari,0.682927


In [274]:
[s for s in vol1_species if s.split()[0] == 'Gypsophila']

['Gypsophila anatolica',
 'Gypsophila antari',
 'Gypsophila arabica',
 'Gypsophila aucheri',
 'Gypsophila damascena',
 'Gypsophila filicaulis']

In [281]:
vol1_index_df[vol1_index_df['mouterde_genus'] == 'Juniperus']
# Inconsistencies between my local vol1_index and the google doc versio

,Unnamed: 0,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon_rank,taxon_rank_detailed
760,2962,Juniperus,NaN,NaN,NaN,genus,genus
761,2965,Juniperus,excelsa,NaN,M. Bieb.,epithet,epithet
762,2968,Juniperus,foetidissima,NaN,Willd.,epithet,epithet
763,2974,Juniperus,oxycedrus,NaN,L.,epithet,epithet


In [263]:
vol1_word_df[vol1_word_df['word_pairs'] == 'Notboîaena marantae']
#should be improved using decode?

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_origin,span_bbox,word_num,word,word_bbox,pruned_word,pruned_word_bbox,word_pairs,possible_index_match,match_score
219622,1,83,7,13,"(65.27999877929688, 124.90997314453125, 428.46...",0,0,"(1.0, 0.0)","(65.27999877929688, 124.90997314453125, 428.46...",0,...,"(65.27999877929688, 133.15997314453125)","(65.27999877929688, 124.90997314453125, 174.44...",0,Notboîaena,"(65.27999877929688, 124.90997314453125, 123.24...",Notboîaena,"(65.27999877929688, 124.90997314453125, 123.24...",Notboîaena marantae,Notholaena marantae,0.894737


In [264]:
vol1_word_df[vol1_word_df['word_pairs'] == 'Ceterack officinarum']

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_origin,span_bbox,word_num,word,word_bbox,pruned_word,pruned_word_bbox,word_pairs,possible_index_match,match_score
230849,1,87,12,27,"(58.31999969482422, 243.67001342773438, 420.62...",0,0,"(1.0, 0.0)","(58.31999969482422, 243.67001342773438, 420.62...",0,...,"(58.31999969482422, 251.92001342773438)","(58.31999969482422, 243.67001342773438, 167.05...",0,Ceterack,"(58.31999969482422, 243.67001342773438, 103.68...",Ceterack,"(58.31999969482422, 243.67001342773438, 103.68...",Ceterack officinarum,Geterach officinarum,0.9


In [265]:
vol1_word_df[vol1_word_df['word_pairs'] == 'Typha laxmannii']

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_origin,span_bbox,word_num,word,word_bbox,pruned_word,pruned_word_bbox,word_pairs,possible_index_match,match_score
262937,1,100,10,21,"(41.52000045776367, 181.1099853515625, 404.538...",0,0,"(1.0, 0.0)","(41.52000045776367, 181.1099853515625, 404.538...",0,...,"(41.52000045776367, 189.3599853515625)","(41.52000045776367, 181.1099853515625, 127.991...",0,Typha,"(41.52000045776367, 181.1099853515625, 73.7514...",Typha,"(41.52000045776367, 181.1099853515625, 73.7514...",Typha laxmannii,Typha laxmannii,1.0
263579,1,100,18,33,"(37.68000030517578, 311.70001220703125, 404.20...",0,0,"(1.0, 0.0)","(37.68000030517578, 311.70001220703125, 404.20...",4,...,"(205.68260192871094, 319.20001220703125)","(205.68260192871094, 311.70001220703125, 278.4...",9,Typha,"(210.19259643554688, 311.70001220703125, 235.1...",Typha,"(210.19259643554688, 311.70001220703125, 235.1...",Typha laxmannii,Typha laxmannii,1.0


In [266]:
vol1_word_df[vol1_word_df['word_pairs'] == 'Typha australis']

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_origin,span_bbox,word_num,word,word_bbox,pruned_word,pruned_word_bbox,word_pairs,possible_index_match,match_score
262322,1,100,1,3,"(42.0, 40.22998046875, 404.7052917480469, 51.2...",0,0,"(1.0, 0.0)","(42.0, 40.22998046875, 404.7052917480469, 51.2...",0,...,"(42.0, 48.47998046875)","(42.0, 40.22998046875, 74.04135131835938, 51.2...",0,Typha,"(42.0, 40.22998046875, 74.04135131835938, 51.2...",Typha,"(42.0, 40.22998046875, 74.04135131835938, 51.2...",Typha australis,Typha australis,1.0


In [267]:
vol1_index_df[vol1_index_df['mouterde_genus'] == 'Typha']

,Unnamed: 0,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon_rank,taxon_rank_detailed
1348,5348,Typha,NaN,NaN,NaN,genus,genus
1349,5355,Typha,australis,NaN,Schum. et Thonn.,epithet,epithet
1350,5359,Typha,—latifolia,NaN,L.,epithet,epithet
1351,5361,Typha,laxmannii,NaN,Lepechin,epithet,epithet


In [268]:
likely_results.loc[i, 'word']

'Scleranthus'

In [193]:
page.add_highlight_annot(r_high)
annot_high.set_colors(stroke=getColor("blue"))

marked_epithet_fname = "../output/local/main_text/3fuzzyMatch90_vol1.pdf"
vol1_doc.save(marked_epithet_fname, garbage=4, clean=True)

In [192]:
annot_high.set_colors(stroke=getColor("blue"))

In [164]:
page = vol1_doc[5]
page.search_for("L'")

[]

In [155]:
marked_epithet_fname = "../output/local/main_text/fuzzyMatch90_vol1.pdf"
vol1_doc.save(marked_epithet_fname, garbage=4, clean=True)